In [1]:
%load_ext autoreload
%autoreload 2
from dimcat import (
    Corpus,  
    Pipeline,
    IsAnnotatedFilter,
    CorpusGrouper, 
    PieceGrouper, 
    ModeGrouper, 
    ChordSymbolBigrams, 
    ChordSymbolUnigrams,
    LocalKeySlicer,
)
import pandas as pd

In [2]:
corpus_path = "~/all_subcorpora"
corpus = Corpus(directory=corpus_path)
corpus.data

INFO     Parse -- parse.py (line 1920) parse_tsv():
	The file dataframe.tsv was recognized to contain labels but no label column 'label' was found in ['Unnamed: 0', 'composer', 'piece', 'title', 'annotator', 'meter', 'tempo', 'measure', 'beat', 'chord', 'altchord', 'cadence', 'pedal', 'numeral', 'form', 'figbass', 'changes', 'relativeroot', 'pedalend', 'phraseend', 'global_key', 'local_key']
	    Specify parse_tsv(key='medtner_tales', cols={'label'=label_column_name}).


7206 files.
KEY                        -> EXTENSIONS
----------------------------------------
ABC                        -> {'.mscx': 70, '.tsv': 211}
bach_en_fr_suites          -> {'.mscx': 89, '.tsv': 268}
bach_solo                  -> {'.mscx': 68, '.tsv': 205}
beethoven_piano_sonatas    -> {'.mscx': 87, '.tsv': 238}
chopin_mazurkas            -> {'.mscx': 55, '.tsv': 166}
corelli                    -> {'.mscx': 149, '.tsv': 436}
couperin_clavecin          -> {'.mscx': 13, '.tsv': 40}
couperin_concerts          -> {'.mscx': 91, '.tsv': 267}
debussy_suite_bergamasque  -> {'.mscx': 4, '.tsv': 13}
dvorak_silhouettes         -> {'.mscx': 12, '.tsv': 37}
frescobaldi_fiori_musicali -> {'.mscx': 48, '.tsv': 145}
gastoldi_baletti           -> {'.mscx': 19, '.tsv': 40}
grieg_lyrical_pieces       -> {'.mscx': 66, '.tsv': 199}
jazz_transcriptions        -> {'.mscx': 293, '.tsv': 563}
jc_bach_sonatas            -> {'.mscx': 29, '.tsv': 75}
kleine_geistliche_konzerte -> {'.mscx': 56, '.tsv': 168

In [3]:
annotated = IsAnnotatedFilter().process_data(corpus)
print(f"Before: {len(corpus.indices[()])} IDs, after filtering: {len(annotated.indices[()])}")

Before: 1998 IDs, after filtering: 1061


# Tokens

All symbols, independent of the local key (the mode of which changes their semantics).

In [4]:
for group, dict_of_dfs in annotated.iter_facet("expanded", concatenate=True):
    for tuple_of_all_id_tuples, all_annotations in dict_of_dfs.items():
        pass
all_annotations.sample(20)

INFO     bach_en_fr_suites_BWV807_07_Gigue -- transformations.py (line 261) dfs2quarterbeats():
	Piece contains third endings, note that only second endings are taken into account.
WARNING  chopin_mazurkas_BI157-3op59-3 -- utils.py (line 1485) make_interval_index():
	Cannot create IntervalIndex from these breaks:
	[0.0, 1.0, 4.0, 5.0, 8.0, 10.0, 11.0, 13.0, 14.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 25.0, 28.0, 29.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 40.0, 41.0, 43.0, 46.0, 48.0, 49.0, 52.0, 53.0, 54.0, 55.0, 58.0, 60.0, 61.0, 63.0, 64.0, 65.0, 66.0, 67.0, 70.0, 72.0, 73.0, 76.0, 77.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 97.0, 100.0, 101.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 111.0, 112.0, 113.0, 114.0, 115.0, 117.0, 118.0, 119.0, 120.0, 121.0, 123.0, 124.0, 125.0, 128.0, 129.5, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 142.0, 142.5, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 1

quarterbeats  \
kozeluh_sonatas         26op26no2a                        [254.5, 256.5)          509/2   
bach_solo               BWV1001_04_Presto                 [36.0, 37.5)               36   
mendelssohn_quartets    01op12a                           [456.0, 460.0)            456   
bach_en_fr_suites       BWV815_04_Gavotte                 [60.0, 61.0)               60   
bach_solo               BWV1006_05_Bourée                 [123.0, 125.0)            123   
schubert_winterreise    n15                               [31.5, 32.0)             63/2   
mendelssohn_quartets    03op44,1a                         [1204.0, 1206.0)         1204   
couperin_concerts       c10n03_plainte                    [103.5, 104.5)          207/2   
ABC                     n05op18-5_03                      [258.5, 262.5)          517/2   
                        n11op95_04                        [221.0, 221.5)            221   
couperin_concerts       c07n01_grave                      [36.0, 37.5)               36   
kozeluh_sonatas         28op30no1b                        [50.5, 51.0)            101/2   
jc_bach_sonatas         wa04op05no4b_Rondeaux             [98.0, 99.0)               98   
bach_solo               BWV1001_02_Fuga                   [236.0, 236.5)            236   
beethoven_piano_sonatas 21-1                              [192.0, 193.0)            192   
corelli                 op03n07d                          [63.0, 64.5)               63   
kozeluh_sonatas         15op13no3c                        [274.5, 276.0)          549/2   
bach_en_fr_suites       BWV814_02_Courante                [33.5, 35.5)             67/2   
                        BWV815_07_Gigue                   [93.0, 94.0)               93   
liszt_pelerinage        161.07_Apres_une_lecture_du_Dante 116                       268   

                                                                           duration_qb  \
kozeluh_sonatas         26op26no2a                        [254.5, 256.5)           2.0   
bach_solo               BWV1001_04_Presto                 [36.0, 37.5)             1.5   
mendelssohn_quartets    01op12a                           [456.0, 460.0)           4.0   
bach_en_fr_suites       BWV815_04_Gavotte                 [60.0, 61.0)             1.0   
bach_solo               BWV1006_05_Bourée                 [123.0, 125.0)           2.0   
schubert_winterreise    n15                               [31.5, 32.0)             0.5   
mendelssohn_quartets    03op44,1a                         [1204.0, 1206.0)         2.0   
couperin_concerts       c10n03_plainte                    [103.5, 104.5)           1.0   
ABC                     n05op18-5_03                      [258.5, 262.5)           4.0   
                        n11op95_04                        [221.0, 221.5)           0.5   
couperin_concerts       c07n01_grave                      [36.0, 37.5)             1.5   
kozeluh_sonatas         28op30no1b                        [50.5, 51.0)             0.5   
jc_bach_sonatas         wa04op05no4b_Rondeaux             [98.0, 99.0)             1.0   
bach_solo               BWV1001_02_Fuga                   [236.0, 236.5)           0.5   
beethoven_piano_sonatas 21-1                              [192.0, 193.0)           1.0   
corelli                 op03n07d                          [63.0, 64.5)             1.5   
kozeluh_sonatas         15op13no3c                        [274.5, 276.0)           1.5   
bach_en_fr_suites       BWV814_02_Courante                [33.5, 35.5)             2.0   
                        BWV815_07_Gigue                   [93.0, 94.0)             1.0   
liszt_pelerinage        161.07_Apres_une_lecture_du_Dante 116                      NaN   

                                                                             mc  \
kozeluh_sonatas         26op26no2a                        [254.5, 256.5)     87   
bach_solo               BWV1001_04_Presto                 [36.0, 37.5)       25   
mendelssohn_quartets   

In [5]:
no_chord = all_annotations.chord.isna()
print(f"Concatenated annotation tables contains {all_annotations.shape[0]} rows. {no_chord.sum()} of them are not chords. Their values are:")
all_annotations[no_chord].value_counts()

Concatenated annotation tables contains 210514 rows. 1281 of them are not chords. Their values are:


Series([], dtype: int64)

In [6]:
all_chords = all_annotations[~no_chord]
print(f"Corpus contains {all_chords.shape[0]} tokens and {len(all_chords.chord.unique())} types over {len(tuple_of_all_id_tuples)} documents.")

Corpus contains 209233 tokens and 4599 types over 1061 documents.


In [7]:
#from ms3 import write_tsv
#write_tsv(all_annotations[all_annotations.pedalend.notna()], './issues/pedalpoints.tsv', pre_process=False)

# Unigrams
For computing unigram statistics, the tokens need to be grouped by their occurrence within a major or a minor key because this changes their meaning. To that aim, the annotated corpus needs to be sliced into contiguous localkey segments which are then grouped into a major (`is_minor=False`) and a minor group.

In [8]:
localkey_slices = LocalKeySlicer().process_data(annotated)

INFO     bach_en_fr_suites_BWV807_07_Gigue -- transformations.py (line 261) dfs2quarterbeats():
	Piece contains third endings, note that only second endings are taken into account.
WARNING  chopin_mazurkas_BI157-3op59-3 -- utils.py (line 1485) make_interval_index():
	Cannot create IntervalIndex from these breaks:
	[0.0, 1.0, 4.0, 5.0, 8.0, 10.0, 11.0, 13.0, 14.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 25.0, 28.0, 29.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 40.0, 41.0, 43.0, 46.0, 48.0, 49.0, 52.0, 53.0, 54.0, 55.0, 58.0, 60.0, 61.0, 63.0, 64.0, 65.0, 66.0, 67.0, 70.0, 72.0, 73.0, 76.0, 77.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 97.0, 100.0, 101.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 111.0, 112.0, 113.0, 114.0, 115.0, 117.0, 118.0, 119.0, 120.0, 121.0, 123.0, 124.0, 125.0, 128.0, 129.5, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 142.0, 142.5, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 1

In [9]:
mode_slices = ModeGrouper().process_data(localkey_slices)

In [10]:
unigrams = ChordSymbolUnigrams(once_per_group=True).process_data(mode_slices)

Group (True,) is missing 'expanded' for the following indices:
[('couperin_concerts', 'c08n08_sarabande', Interval(72.0, 72.0, closed='left'))]


## Whole dataset

In [11]:
unigrams.get()

localkey_is_minor  IDs        chord       
False              group_ids  I               19609
                              V                9342
                              I6               8134
                              V7               7947
                              IV               5065
                                              ...  
True               group_ids  ii64(6)/iv          1
                              iio64(+5)/iv        1
                              #viio43(#5)         1
                              #viio65(#5)         1
                              vio                 1
Name: count, Length: 5361, dtype: int64

In [12]:
modes = {True: 'MINOR', False: 'MAJOR'}
for (is_minor,), ugs in unigrams.iter():
    print(f"{modes[is_minor]} UNIGRAMS\n{ugs.shape[0]} types, {ugs.sum()} tokens")
    print(ugs.head(20).to_string())

MAJOR UNIGRAMS
2860 types, 121815 tokens
IDs        chord
group_ids  I        19609
           V         9342
           I6        8134
           V7        7947
           IV        5065
           vi        3173
           ii6       3136
           V(64)     3061
           ii        2875
           V6        2858
           V65       2517
           V43       2385
           IV6       2340
           V2        2276
           viio6     1627
           I64       1382
           viio      1090
           V(4)       925
           ii65       919
           vi6        915
MINOR UNIGRAMS
2501 types, 83567 tokens
IDs        chord 
group_ids  i         11457
           V          6675
           i6         4499
           V7         3694
           iv         2993
           iv6        1829
           VI         1741
           V(64)      1712
           III        1682
           v          1449
           V6         1441
           V65        1200
           V43         986
           VI

## Per corpus

In [13]:
corpus_wise_unigrams = Pipeline([CorpusGrouper(), ChordSymbolUnigrams(once_per_group=True)]).process_data(mode_slices)

Group ('couperin_concerts', True) is missing 'expanded' for the following indices:
[('couperin_concerts', 'c08n08_sarabande', Interval(72.0, 72.0, closed='left'))]


In [14]:
corpus_wise_unigrams.get()

corpus           localkey_is_minor  IDs        chord   
ABC              False              group_ids  I           3401
                                               V7          1622
                                               I6          1489
                                               V           1223
                                               IV           755
                                                           ... 
wf_bach_sonatas  True               group_ids  viio/V         1
                                               v(#9)          1
                                               V2/V/III       1
                                               III(9)         1
                                               iv(4)          1
Name: count, Length: 16052, dtype: int64

In [15]:
for (corpus, is_minor), ugs in corpus_wise_unigrams.iter():
    print(f"{corpus} {modes[is_minor]} unigrams ({ugs.shape[0]} types, {ugs.sum()} tokens)")
    print(ugs.head(5).to_string())

ABC MAJOR unigrams (789 types, 20101 tokens)
IDs        chord
group_ids  I        3401
           V7       1622
           I6       1489
           V        1223
           IV        755
ABC MINOR unigrams (544 types, 7857 tokens)
IDs        chord
group_ids  i        1201
           V         665
           V7        559
           i6        491
           iv        249
bach_en_fr_suites MAJOR unigrams (400 types, 5315 tokens)
IDs        chord
group_ids  I        531
           V        439
           I6       317
           IV       218
           V6       193
bach_en_fr_suites MINOR unigrams (579 types, 7757 tokens)
IDs        chord
group_ids  i        698
           i6       417
           V        391
           iv       318
           III      200
bach_solo MAJOR unigrams (269 types, 3195 tokens)
IDs        chord
group_ids  I        435
           V        324
           I6       231
           V7       158
           IV       143
bach_solo MINOR unigrams (285 types, 3827 tokens)


## Per piece

In [16]:
piece_wise_unigrams = Pipeline([PieceGrouper(), ChordSymbolUnigrams(once_per_group=True)]).process_data(mode_slices)

Group ('c08n08_sarabande', True) is missing 'expanded' for the following indices:
[('couperin_concerts', 'c08n08_sarabande', Interval(72.0, 72.0, closed='left'))]


# Bigrams

In [18]:
bigrams = ChordSymbolBigrams(once_per_group=True).process_data(mode_slices)

AssertionError: Cannot concat indices that do not have the same number of levels

## Whole dataset

In [ ]:
unigrams.get()

In [ ]:
modes = {True: 'MINOR', False: 'MAJOR'}
for (is_minor,), ugs in unigrams.iter():
    print(f"{modes[is_minor]} UNIGRAMS\n{ugs.shape[0]} types, {ugs.sum()} tokens")
    print(ugs.head(20).to_string())

## Per corpus

In [ ]:
corpus_wise_unigrams = Pipeline([CorpusGrouper(), ChordSymbolUnigrams(once_per_group=True)]).process_data(mode_slices)

In [ ]:
corpus_wise_unigrams.get()

In [ ]:
for (corpus, is_minor), ugs in corpus_wise_unigrams.iter():
    print(f"{corpus} {modes[is_minor]} unigrams ({ugs.shape[0]} types, {ugs.sum()} tokens)")
    print(ugs.head(5).to_string())

## Per piece

In [ ]:
piece_wise_unigrams = Pipeline([PieceGrouper(), ChordSymbolUnigrams(once_per_group=True)]).process_data(mode_slices)

In [ ]:
piece_wise_unigrams.get()

In [17]:
piece_wise_unigrams.get()

fname                    localkey_is_minor  IDs        chord  
00_allemande             True               group_ids  V          8
                                                       i          6
                                                       i6         5
                                                       V7         5
                                                       III        4
                                                                 ..
wa06op05no6c_Allegretto  True               group_ids  v6(#72)    1
                                                       V2/v       1
                                                       V65/v      1
                                                       iio/v      1
                                                       V2         1
Name: count, Length: 57844, dtype: int64